<a href="https://colab.research.google.com/github/vasudevgupta7/gsoc-wav2vec2/blob/main/notebooks/librispeech_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!git clone https://github.com/vasudevgupta7/gsoc-wav2vec2 && cd gsoc-wav2vec2 && pip3 install -r requirements.txt

In [2]:
!wget https://www.openslr.org/resources/12/test-clean.tar.gz -P /content/gsoc-wav2vec2/data && tar -xf /content/gsoc-wav2vec2/data/test-clean.tar.gz -C /content/gsoc-wav2vec2/data/
!wget https://www.openslr.org/resources/12/test-other.tar.gz -P /content/gsoc-wav2vec2/data && tar -xf /content/gsoc-wav2vec2/data/test-other.tar.gz -C /content/gsoc-wav2vec2/data/

--2021-06-14 10:57:58--  https://www.openslr.org/resources/12/test-clean.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 346663984 (331M) [application/x-gzip]
Saving to: ‘/content/gsoc-wav2vec2/data/test-clean.tar.gz’

test-clean.tar.gz   100%[===================>] 330.60M   165MB/s    in 2.0s    

2021-06-14 10:58:00 (165 MB/s) - ‘/content/gsoc-wav2vec2/data/test-clean.tar.gz’ saved [346663984/346663984]

--2021-06-14 10:58:04--  https://www.openslr.org/resources/12/test-other.tar.gz
Resolving www.openslr.org (www.openslr.org)... 46.101.158.64
Connecting to www.openslr.org (www.openslr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328757843 (314M) [application/x-gzip]
Saving to: ‘/content/gsoc-wav2vec2/data/test-other.tar.gz’

test-other.tar.gz   100%[===================>] 313.53M   16

In [4]:
import os
os.chdir("/content/gsoc-wav2vec2/notebooks/")

In [5]:
import sys
sys.path.append("../src")

from wav2vec2 import Wav2Vec2Processor, Wav2Vec2ForCTC
import tensorflow as tf

In [6]:
MODEL_ID = "vasudevgupta/tf-wav2vec2-base-960h"

processor = Wav2Vec2Processor(is_tokenizer=False)
tokenizer = Wav2Vec2Processor(is_tokenizer=True)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID)

Total number of loaded variables: 212


In [29]:
from data_utils import DataLoader
from tqdm.auto import tqdm

@tf.function(autograph=True, jit_compile=True)
def tf_forward(*args, **kwargs):
  tf_out = model(*args, **kwargs)["logits"]
  return tf.squeeze(tf.argmax(tf_out, axis=-1))

def infer_librispeech(data_dir, audio_maxlen=500000):
  predictions = []
  labels = []
  tr_dataset_fn = DataLoader(data_dir=data_dir, dataset="librispeech", batch_size=16, audio_maxlen=audio_maxlen)
  tr_dataset = tr_dataset_fn(shuffle=False)
  for batch in tqdm(tr_dataset, total=len(tr_dataset), desc="LibriSpeech Inference ... "):
    speech, label = batch
    tf_out = tf_forward(speech)
    predictions.extend([tokenizer.decode(pred) for pred in tf_out.numpy().tolist()])
    labels.extend([s.decode() for s in label.numpy().tolist()])
  return predictions, labels

In [30]:
predictions, labels = infer_librispeech(data_dir="../data/LibriSpeech/test-clean")
list(zip(predictions, labels))

DISCARDING 2 samples
LOADED 2618 FILES FROM ../data/LibriSpeech/test-clean


[('WE MIGHT PROVISIONALLY THOUGH PERHAPS NOT QUITE CORRECTLY DEFINE MEMORY AS THAT WAY OF KNOWING ABOUT THE PAST WHICH HAS NO ANALOGUE IN OUR KNOWLEDGE OF THE FUTURE SUCH A DEFINITION WOULD AT LEAST SERVE TO MARK THE PROBLEM WITH WHICH WE ARE CONCERNED THOUGH SOME EXPECTATIONS MAY DESERVE TO RANK WITH MEMORY AS REGARDS IMMEDIACY',
  'WE MIGHT PROVISIONALLY THOUGH PERHAPS NOT QUITE CORRECTLY DEFINE MEMORY AS THAT WAY OF KNOWING ABOUT THE PAST WHICH HAS NO ANALOGUE IN OUR KNOWLEDGE OF THE FUTURE SUCH A DEFINITION WOULD AT LEAST SERVE TO MARK THE PROBLEM WITH WHICH WE ARE CONCERNED THOUGH SOME EXPECTATIONS MAY DESERVE TO RANK WITH MEMORY AS REGARDS IMMEDIACY'),
 ('SIMMONS TWO BOOKS MENTIONED IN AN EARLIER  LECTURE DO NOT TOUCH KNOWLEDGE MEMORY AT ALL CLOSELY',
  "SEMON'S TWO BOOKS MENTIONED IN AN EARLIER LECTURE DO NOT TOUCH KNOWLEDGE MEMORY AT ALL CLOSELY"),
 ('COME NOW TO THE OTHER CHARACTERISTIC WHICH MEMORY IMAGES MUST HAVE IN ORDER TO ACCOUNT FOR OUR KNOWLEDGE OF THE PAST',
  'I COME

In [ ]:
from jiwer import wer
err = wer(labels, predictions)
err

In [27]:
!nvidia-smi

Mon Jun 14 11:08:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    76W / 149W |   8949MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
# def evaluate_wav(wav_file_path, txt_file_path):
#   batch, _ = tf.audio.decode_wav(tf.io.read_file(wav_file_path))
#   batch = tf.transpose(batch, perm=(1, 0))
#   batch = processor(batch)[:, :45000]
#   tf_out = tf_forward(batch)["logits"]
#   return tokenizer.decode(tf_out.numpy().tolist())

# file_tuple = [("../data/SA2.wav", "../data/SA2.TXT"), ("../data/sample.wav", "../data/sample.TXT")]
# predictions_map = map(lambda x: evaluate(*x), file_tuple)
# list(predictions_map)